## File Renaming

In [22]:
import os
from tqdm import tqdm

# --- 1. CONFIGURATION ---

# Path to your source dataset that has subfolders like 'Loudspeaker', 'Window', etc.
# This is the directory where the folders you want to process are located.
ROOT_DIRECTORY = "/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset/"

# --- 2. SCRIPT LOGIC ---

print(f"Starting file renaming process in root directory: {ROOT_DIRECTORY}")

# A list to keep track of all files to be renamed for the progress bar
files_to_rename = []

# First, walk the directory to find all files that need renaming
for dirpath, _, filenames in os.walk(ROOT_DIRECTORY):
    for filename in filenames:
        # Check if the file is an image and follows the 'frame_...' pattern
        if filename.lower().startswith('frame_') and filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            full_path = os.path.join(dirpath, filename)
            files_to_rename.append(full_path)

if not files_to_rename:
    print("No files matching the 'frame_...' pattern were found. Exiting.")
    exit()
    
print(f"Found {len(files_to_rename)} files to rename.")

# Now, iterate through the found files and rename them
for old_path in tqdm(files_to_rename, desc="Renaming files"):
    try:
        # Get the directory path and the original filename
        dirpath, old_filename = os.path.split(old_path)
        
        # Get the name of the parent folder
        folder_name = os.path.basename(dirpath)
        
        # Create the new filename
        new_filename = f"{folder_name}_{old_filename}"
        
        # Create the full new path
        new_path = os.path.join(dirpath, new_filename)
        
        # Rename the file
        os.rename(old_path, new_path)

    except Exception as e:
        print(f"\nError renaming {old_path}. Reason: {e}")

print("\n--- File renaming process finished successfully! ---")

Starting file renaming process in root directory: /mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset/
Found 7195 files to rename.


Renaming files: 100%|██████████| 7195/7195 [00:18<00:00, 389.66it/s]


--- File renaming process finished successfully! ---


## Vision Transformer For Annotation

In [1]:
import os
import cv2
import torch
import shutil
import gc
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import supervision as sv
from tqdm import tqdm
import numpy as np

# --- 1. CONFIGURATION ---

# Choose the Owl-ViT model. 'base-patch32' is a good starting point.
# For higher accuracy (and higher VRAM usage), you can use 'large-patch14'.
MODEL_NAME = "google/owlvit-base-patch32"

# Set a max size for images to manage memory usage.
MAX_IMAGE_SIZE = 1280

# Path to your source dataset (which contains the renamed files like 'LoudSpeaker_frame_XXXX.jpg')
SOURCE_DATA_PATH = "/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset/"

# Path where the new, YOLOv8-formatted dataset will be created.
OUTPUT_DATA_PATH = "/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset_for_YOLOv8/"

# --- MODIFICATION: Use a single, generic prompt to find the main object ---
# This is ideal for images with one primary object. The model will focus on finding
# the best bounding box, and we will assign a placeholder class ID of 0.
CLASSES = ["the main object in the photo"]

# Owl-ViT confidence threshold. You can adjust this as needed.
CONFIDENCE_THRESHOLD = 0.1

# --- 2. SETUP & MODEL LOADING ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {DEVICE}")
print(f"Using Model: {MODEL_NAME}")

# Create and print the class mapping for user verification
class_name_to_id = {name: i for i, name in enumerate(CLASSES)}
print("\n--- Class to ID Mapping ---")
print(class_name_to_id)
print("---------------------------\n")


print("Loading Owl-ViT model and processor...")
try:
    processor = AutoProcessor.from_pretrained(MODEL_NAME)
    model = AutoModelForZeroShotObjectDetection.from_pretrained(MODEL_NAME).to(DEVICE)
    print("Model loaded successfully.")
except Exception as e:
    print(f"FATAL ERROR: Failed to load model. Reason: {e}")
    exit()

# --- 3. CREATE YOLOv8 DIRECTORY STRUCTURE ---
output_images_path = os.path.join(OUTPUT_DATA_PATH, "images")
output_labels_path = os.path.join(OUTPUT_DATA_PATH, "labels")
os.makedirs(output_images_path, exist_ok=True)
os.makedirs(output_labels_path, exist_ok=True)
print(f"Output directories created at: {OUTPUT_DATA_PATH}")

# --- 4. FIND ALL IMAGES RECURSIVELY ---
print("Scanning for all images...")
all_image_paths = []
for root, _, files in os.walk(SOURCE_DATA_PATH):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            all_image_paths.append(os.path.join(root, file))
print(f"Found {len(all_image_paths)} total images to process.")


# --- 5. HELPER FUNCTION FOR IMAGE RESIZING ---
def resize_image(image, max_size):
    """
    Resizes an image to a maximum size while maintaining aspect ratio.
    """
    h, w, _ = image.shape
    if max(h, w) > max_size:
        if h > w:
            new_h = max_size
            new_w = int(w * (max_size / h))
        else:
            new_w = max_size
            new_h = int(h * (max_size / w))
        return cv2.resize(image, (new_w, new_h))
    return image

# --- 6. MAIN ANNOTATION LOOP ---
for source_image_path in tqdm(all_image_paths, desc="Annotating with Owl-ViT"):
    try:
        image_filename = os.path.basename(source_image_path)
        dest_image_path = os.path.join(output_images_path, image_filename)
        label_filename = os.path.splitext(image_filename)[0] + ".txt"
        dest_label_path = os.path.join(output_labels_path, label_filename)

        original_image = cv2.imread(source_image_path)
        if original_image is None:
            print(f"\nWarning: Could not read image {source_image_path}. Skipping.")
            continue
        
        original_h, original_w, _ = original_image.shape

        resized_image = resize_image(original_image, MAX_IMAGE_SIZE)
        
        # Owl-ViT expects a list of text queries
        text_queries = [CLASSES]
        
        # Run inference
        inputs = processor(text=text_queries, images=resized_image, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            outputs = model(**inputs)

        # Process results
        resized_h, resized_w, _ = resized_image.shape
        target_sizes = torch.tensor([[resized_h, resized_w]]).to(DEVICE)
        
        # Note: Owl-ViT uses 'post_process_object_detection'
        results = processor.post_process_object_detection(outputs=outputs, target_sizes=target_sizes, threshold=CONFIDENCE_THRESHOLD)

        # The results are per-image, so we take the first one (index 0)
        result = results[0]
        boxes = result["boxes"]
        scores = result["scores"]
        labels = result["labels"]

        if len(scores) > 0:
            shutil.copy(source_image_path, dest_image_path)

            # --- Find and process only the single best detection ---
            # Find the index of the detection with the highest confidence score
            best_detection_index = torch.argmax(scores)

            # Get the single best box
            best_box = boxes[best_detection_index]
            
            # Since we use a generic prompt, the label_id will always be 0.
            # We will use this as a placeholder. Your label corrector script will fix it.
            placeholder_label_id = 0
            
            # Box is already in xyxy format
            x1, y1, x2, y2 = best_box.tolist()

            # Scale the bounding box from the resized image back to the original image's dimensions
            x1_orig = x1 * (original_w / resized_w)
            y1_orig = y1 * (original_h / resized_h)
            x2_orig = x2 * (original_w / resized_w)
            y2_orig = y2 * (original_h / resized_h)

            # Convert to YOLO format and normalize
            box_yolo_x = ((x1_orig + x2_orig) / 2) / original_w
            box_yolo_y = ((y1_orig + y2_orig) / 2) / original_h
            box_yolo_w = (x2_orig - x1_orig) / original_w
            box_yolo_h = (y2_orig - y1_orig) / original_h

            yolo_formatted_annotation = (
                f"{int(placeholder_label_id)} {box_yolo_x:.6f} {box_yolo_y:.6f} {box_yolo_w:.6f} {box_yolo_h:.6f}"
            )

            with open(dest_label_path, "w") as f:
                f.write(yolo_formatted_annotation + "\n")

    except Exception as e:
        import traceback
        print(f"\n---!!! UNEXPECTED ERROR processing {source_image_path} !!!---")
        traceback.print_exc()
        print(f"Error Type: {type(e).__name__}, Message: {e}")
        print("-----------------------------------------------------------------")
    finally:
        # --- MEMORY CLEANUP ---
        if 'inputs' in locals(): del inputs
        if 'outputs' in locals(): del outputs
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

print("\n--- Owl-ViT Annotation process finished! ---")
print(f"Your YOLOv8-ready dataset is located at: {OUTPUT_DATA_PATH}")


Using Device: cuda
Using Model: google/owlvit-base-patch32

--- Class to ID Mapping ---
{'the main object in the photo': 0}
---------------------------

Loading Owl-ViT model and processor...
Model loaded successfully.
Output directories created at: /mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset_for_YOLOv8/
Scanning for all images...
Found 168 total images to process.


Annotating with Owl-ViT: 100%|██████████| 168/168 [00:42<00:00,  3.99it/s]


--- Owl-ViT Annotation process finished! ---
Your YOLOv8-ready dataset is located at: /mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset_for_YOLOv8/


## Visualization of Bounding Boxes

In [6]:
import os
from tqdm import tqdm

# --- 1. CONFIGURATION ---

# Path to the YOLOv8 dataset directory that contains the label files.
LABELS_DIRECTORY = "/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset_for_YOLOv8/labels/"

# Your master class list. This is the ground truth for mapping names to IDs.
# Ensure this list is in the final order you want for YOLOv8 training.
# Note: The user's prompt provided a different list, so I've used that one. 
# Please verify this is correct.
CLASSES = ["unit", "speaker", "motor", "assembly"]


# --- 2. SCRIPT LOGIC ---

print("--- Starting Dynamic YOLOv8 Label Correction Script ---")

# Create the master mapping from class name to the correct ID
class_name_to_id = {name: i for i, name in enumerate(CLASSES)}
print("Master Class-to-ID mapping has been created:")
print(class_name_to_id)

# Get all the label files from the directory
try:
    label_files = [f for f in os.listdir(LABELS_DIRECTORY) if f.endswith('.txt')]
    if not label_files:
        print(f"Error: No .txt files found in {LABELS_DIRECTORY}. Exiting.")
        exit()
except FileNotFoundError:
    print(f"Error: The directory was not found: {LABELS_DIRECTORY}. Please check the path.")
    exit()

print(f"\nFound {len(label_files)} label files to process.")

# Initialize a counter for corrected files
corrected_files_count = 0

# --- 3. CORRECTION LOOP ---
for filename in tqdm(label_files, desc="Correcting Labels"):
    try:
        # --- DYNAMIC CLASS NAME EXTRACTION ---
        # Parse the filename to get the class name part before "_frame_"
        # Example: "Control Unit_frame_1496.txt" -> "Control Unit"
        
        # We split by '_frame_' and take the first part.
        # This handles class names with spaces correctly.
        parts = filename.split('_frame_')
        if len(parts) < 2:
            print(f"\nWarning: Filename '{filename}' does not match 'classname_frame_xxx' format. Skipping.")
            continue
            
        correct_class_name = parts[0]

        # Check if the extracted class name exists in our master list
        if correct_class_name not in class_name_to_id:
            print(f"\nWarning: Extracted class '{correct_class_name}' from filename '{filename}' is not in the master CLASSES list. Skipping.")
            continue

        # Get the correct class ID from our master mapping
        correct_class_id = class_name_to_id[correct_class_name]
        
        label_path = os.path.join(LABELS_DIRECTORY, filename)
        
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        corrected_lines = []
        was_corrected = False

        for line in lines:
            line_parts = line.strip().split()
            
            if len(line_parts) == 5:
                current_class_id = int(line_parts[0])
                
                # If the ID is already correct, no change is needed.
                # If it's incorrect, we set was_corrected to True.
                if current_class_id != correct_class_id:
                    was_corrected = True
                
                # Rebuild the line with the *correct* class ID, keeping coordinates.
                corrected_line = f"{correct_class_id} {line_parts[1]} {line_parts[2]} {line_parts[3]} {line_parts[4]}"
                corrected_lines.append(corrected_line)
            else:
                corrected_lines.append(line.strip()) # Keep malformed lines
        
        # Only rewrite the file if a correction was actually made
        if was_corrected:
            corrected_files_count += 1
            with open(label_path, 'w') as f:
                f.write("\n".join(corrected_lines))
                if corrected_lines:
                    f.write("\n")

    except Exception as e:
        print(f"\nError processing file {filename}. Reason: {e}")

print("\n--- Label correction process finished! ---")
print(f"Total files checked: {len(label_files)}")
print(f"Total files corrected: {corrected_files_count}")



--- Starting Dynamic YOLOv8 Label Correction Script ---
Master Class-to-ID mapping has been created:
{'unit': 0, 'speaker': 1, 'motor': 2, 'assembly': 3}

Found 168 label files to process.


Correcting Labels: 100%|██████████| 168/168 [00:00<00:00, 391.26it/s]


--- Label correction process finished! ---
Total files checked: 168
Total files corrected: 42


In [7]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# --- 1. CONFIGURATION ---

# Path to the dataset generated by the previous script
DATASET_PATH = "/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset_for_YOLOv8/"

# Directory where the visualized images will be saved
VISUALIZATION_OUTPUT_PATH = "/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/annotated_images/"

# Your class names, in the exact same order as the previous script
CLASSES = ["unit", "speaker", "motor", "assmebly"]

# Optional: Colors for the bounding boxes (in BGR format)
# If you have more classes, add more colors here.
BOX_COLORS = [
    (255, 0, 0),    # Blue for Window_Motor
    (0, 255, 0),    # Green for Control Unit
    (0, 0, 255),    # Red for LoudSpeaker
    (255, 255, 0)   # Cyan for Window
]

# --- 2. SETUP ---
images_dir = os.path.join(DATASET_PATH, "images")
labels_dir = os.path.join(DATASET_PATH, "labels")

# Create the output directory if it doesn't exist
os.makedirs(VISUALIZATION_OUTPUT_PATH, exist_ok=True)
print(f"Visualized images will be saved to: {VISUALIZATION_OUTPUT_PATH}")

# Find all image files
image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

if not image_files:
    print(f"Error: No images found in {images_dir}. Please check the path.")
    exit()

print(f"Found {len(image_files)} images to visualize.")

# --- 3. VISUALIZATION LOOP ---
for image_filename in tqdm(image_files, desc="Visualizing Annotations"):
    try:
        # Define paths
        image_path = os.path.join(images_dir, image_filename)
        label_filename = os.path.splitext(image_filename)[0] + ".txt"
        label_path = os.path.join(labels_dir, label_filename)
        output_image_path = os.path.join(VISUALIZATION_OUTPUT_PATH, image_filename)

        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"\nWarning: Could not read image {image_path}. Skipping.")
            continue

        # Check if a corresponding label file exists
        if not os.path.exists(label_path):
            # If no label, just copy the original image
            cv2.imwrite(output_image_path, image)
            continue

        # Get image dimensions
        img_h, img_w, _ = image.shape

        # Read the YOLO label file
        with open(label_path, 'r') as f:
            for line in f:
                # Parse the line: class_id, x_center, y_center, width, height
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                
                class_id = int(parts[0])
                x_center_norm = float(parts[1])
                y_center_norm = float(parts[2])
                w_norm = float(parts[3])
                h_norm = float(parts[4])

                # Denormalize coordinates to pixel values
                box_w = int(w_norm * img_w)
                box_h = int(h_norm * img_h)
                box_x = int((x_center_norm * img_w) - (box_w / 2))
                box_y = int((y_center_norm * img_h) - (box_h / 2))

                # Get class name and color
                class_name = CLASSES[class_id] if 0 <= class_id < len(CLASSES) else "Unknown"
                color = BOX_COLORS[class_id % len(BOX_COLORS)]

                # Draw the bounding box
                cv2.rectangle(image, (box_x, box_y), (box_x + box_w, box_y + box_h), color, 2)

                # Prepare text for the label
                label_text = f"{class_name}"
                
                # Calculate text size to draw a background rectangle
                (text_width, text_height), baseline = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                
                # Draw a filled rectangle as a background for the text
                cv2.rectangle(image, (box_x, box_y - text_height - 5), (box_x + text_width, box_y), color, -1)
                
                # Put the class name text on the rectangle
                cv2.putText(image, label_text, (box_x, box_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Save the image with drawn annotations
        cv2.imwrite(output_image_path, image)

    except Exception as e:
        import traceback
        print(f"\n---!!! UNEXPECTED ERROR visualizing {image_filename} !!!---")
        traceback.print_exc()
        print(f"Error Type: {type(e).__name__}, Message: {e}")
        print("----------------------------------------------------------")

print("\n--- Visualization process finished! ---")



Visualized images will be saved to: /mnt/c/Users/mohsinali.mirza/Downloads/Grounding/annotated_images/
Found 168 images to visualize.


Visualizing Annotations:   0%|          | 0/168 [00:00<?, ?it/s]

Visualizing Annotations: 100%|██████████| 168/168 [00:02<00:00, 71.64it/s]


--- Visualization process finished! ---


## Create YAML for YOLOv8

In [9]:
import yaml
import os

# --- Configuration ---
# Path to the root of your YOLOv8 dataset
dataset_base_path = '/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset_for_YOLOv8/'

# Your class names in the correct order
class_names = ["control unit", "loud speaker", "Motor", "window regulator assembly"]

# --- Create the YAML content ---
# Note: We use os.path.join to ensure paths are correct.
# YOLOv8 will correctly interpret these paths.
dataset_yaml_content = {
    'path': dataset_base_path,
    'train': os.path.join(dataset_base_path, 'images'),  # Using the same set for train and val for this example
    'val': os.path.join(dataset_base_path, 'images'),    # Ideally, you should split your data into train/val sets
    'nc': len(class_names),
    'names': class_names
}

# --- Write the YAML file ---
# We will save this file in the directory that contains the dataset folder.
output_yaml_path = os.path.join('/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/', 'car_parts.yaml')

try:
    with open(output_yaml_path, 'w') as f:
        yaml.dump(dataset_yaml_content, f, sort_keys=False)
    print(f"Successfully created dataset YAML file at: {output_yaml_path}")
except Exception as e:
    print(f"Error creating YAML file: {e}")


Successfully created dataset YAML file at: /mnt/c/Users/mohsinali.mirza/Downloads/Grounding/car_parts.yaml


## Train Yolov8

In [10]:
from ultralytics import YOLO

# --- 1. Load a Pre-trained Model ---
# 'yolov8n.pt' is the smallest and fastest model, ideal for starting.
# You can also use 'yolov8s.pt', 'yolov8m.pt', etc., for better accuracy at the cost of speed and memory.
model = YOLO('yolov8n.pt')

# --- 2. Define the path to your dataset YAML file ---
dataset_yaml_path = '/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/car_parts.yaml'

# --- 3. Start Fine-Tuning ---
# The training results, including model weights, will be saved in a 'runs/detect/train' directory.
results = model.train(
    data=dataset_yaml_path,
    epochs=100,          # Number of training epochs. Start with 100 and see the results.
    imgsz=640,           # Image size for training. 640 is a common choice.
    batch=8,             # Batch size. Lower this if you run into CUDA memory issues (e.g., to 4 or 2).
    name='yolov8n_car_parts_finetune' # A name for your training run
)

print("\n--- Training finished! ---")
print("Your trained model and results are saved in the 'runs/detect/' directory.")

Ultralytics 8.3.159 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/car_parts.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_car_parts_finetune, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mas

AMP: checks passed ✅
train: Fast image access ✅ (ping: 1.3±0.3 ms, read: 25.5±2.8 MB/s, size: 58.2 KB)


train: Scanning /mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset_for_YOLOv8/labels... 168 images, 0 backgrounds, 0 corrupt: 100%|██████████| 168/168 [00:00<00:00, 856.66it/s]


train: New cache created: /mnt/c/Users/mohsinali.mirza/Downloads/Grounding/Dataset_for_YOLOv8/labels.cache


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

...

  0%|          | 0/21 [00:00<?, ?it/s]UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:233.)
    100/100       1.8G     0.1805     0.2462     0.7937          8        640: 100%|██████████| 21/21 [00:02<00:00,  9.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/11 [00:00<?, ?it/s]UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or 

                   all        168        168      0.998          1      0.995      0.994



100 epochs completed in 0.105 hours.
Optimizer stripped from runs/detect/yolov8n_car_parts_finetune/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8n_car_parts_finetune/weights/best.pt, 6.2MB

Validating runs/detect/yolov8n_car_parts_finetune/weights/best.pt...
Ultralytics 8.3.159 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  5.72it/s]


                   all        168        168      0.998          1      0.995      0.994
          control unit         42         42      0.998          1      0.995      0.995
          loud speaker         42         42      0.998          1      0.995      0.995
                 Motor         42         42      0.999          1      0.995      0.991
window regulator assembly         42         42      0.998          1      0.995      0.995
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/yolov8n_car_parts_finetune

--- Training finished! ---
Your trained model and results are saved in the 'runs/detect/' directory.
